In [22]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
from datetime import datetime 
import os

# definiuję funkcję wyznaczającą rok finansowy - zaczynający się w lipcu i kończący się w czerwcu
def financial_year(date_row): 
    if date_row['month']>=7:
        val=date_row['year']
    else:
        val=date_row['year']-1
    return val

# tworzę tabelę zawierającą odpowiednie informacje
start_date = '2000-04-30' # pierwsza data w bazie nasdaq
end_date = '2025-12-31' # wybrana przeze mnie data końcowa
date_index = pd.date_range(start=start_date, end = end_date)
dates = pd.DataFrame()

dates['full_date'] = date_index
dates['date_id'] = dates['full_date'].apply(lambda x: int(x.strftime("%Y%m%d")))
dates['day'] = dates['full_date'].apply(lambda x: int(x.strftime("%d")))
dates['month'] = dates['full_date'].apply(lambda x: int(x.strftime("%m")))
dates['month_name'] = dates['full_date'].apply(lambda x: x.strftime("%B"))
dates['quarter'] = (dates['month']-1)//3+1 
dates['year'] = dates['full_date'].apply(lambda x: int(x.strftime("%Y")))
dates['financial_year'] = dates.apply(financial_year, axis=1)


# ustanawiam połączenie z bazą
SERVER = 'bigmacindex.database.windows.net'
DATABASE = 'bigmacindex'
DRIVER = 'ODBC Driver 17 for SQL Server'

with open('creds/data_writer_creds.txt','r') as f:
    creds = [line.strip() for line in f.readlines()]

USERNAME = creds[0] 
PASSWORD = creds[1] 


DATABASE_CONNECTION = f'mssql://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER}'

engine = create_engine(DATABASE_CONNECTION)
connection = engine.connect()


# ładuje dane do tabeli dates
dates=dates[['date_id','full_date','day','month','month_name','quarter','year','financial_year']]
dates.to_sql(name='dates',con=connection,if_exists='append', index=False)


# ustawiam odpowiednie indeksy mające na celu przyspieszenie działania funkcji agregujących
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
cursor = cnxn.cursor()


cursor.execute('''CREATE INDEX IX_day ON dates (day)''')
cnxn.commit()

cursor.execute('''CREATE INDEX IX_month ON dates (month)''')
cnxn.commit()

cursor.execute('''CREATE INDEX IX_month_name ON dates (month_name)''')
cnxn.commit()

cursor.execute('''CREATE INDEX IX_quarter ON dates (quarter)''')
cnxn.commit()

cursor.execute('''CREATE INDEX IX_year ON dates (year)''')
cnxn.commit()

cursor.execute('''CREATE INDEX IX_financial_year ON dates (financial_year)''')
cnxn.commit()

cursor.close()
cnxn.close()
